# From $\Lambda$CDM to EDE -- Hands-on Session 2


CosmoVerse Summer School Corfu -- May 13 - 18, 2024

Author: Laura Herold

In this notebook, we are going to compute CMB power spectra and compare them to real data sets. We are going to explore how Early Dark Energy affects cosmological observables and how it could resolve the Hubble tension.

**Requirements:**
- Install `CLASS_EDE` following the instructions on [github](https://github.com/mwt5345/class_ede).
- For plotting, latex is useful but not strictly necessary

# Part 1 - $\Lambda$CDM

First of all, let's import the main python packages that we will need:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from classy import Class
%matplotlib inline
%config InlineBackend.figure_format='retina'

**EDE**:

This time we will use a modified version of [CLASS](https://lesgourg.github.io/class_public/class.html), which allows to model early dark energy.

There are two codes on github that allow to do this:
- [AxiCLASS](https://github.com/PoulinV/AxiCLASS),
- [CLASS_EDE](https://github.com/mwt5345/class_ede/tree/master).

We will use CLASS_EDE here.

We can use CLASS_EDE in the same way as CLASS. If we do not set the EDE parameters, all quantities are computed in the $\Lambda$CDM model. In the first part of this notebook, we will do exactly that: we will compute the CMB power spectra within $\Lambda$CDM. Later, in part 2, we will compare our results to the EDE model.

Let's create an instance of the class "Class":

In [ ]:
LCDM = Class()

Let's fix the values for the $\Lambda$CDM parameters and some technical parameters in CLASS. We fix them to the $\Lambda$CDM mean parameters obtained in the [*Planck* 2018 paper](https://arxiv.org/pdf/1807.06209.pdf) (second column of Table 1).

As mentioned earlier, we do not set the EDE parameters here, hence every quantity is computed within $\Lambda$CDM.

In [ ]:
LCDM.set({'omega_b':0.02237,
          'omega_cdm':0.1200,
          'h':67.36/100.,
          'ln10^{10}A_s':3.044,
          'n_s':0.9649,
          'tau_reio':0.0544})
LCDM.set({'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':3.0})

Let's compute some CMB power spectra! The CMB power spectrum is usually quoted as $C_\ell$'s, which give the variance in the CMB fluctuations at the multipole $\ell$.

In [ ]:
LCDM.compute()
cls_LCDM = LCDM.lensed_cl(2500)
print(cls_LCDM.keys())

There are power spectra in temperature (TT), E-mode and B-mode polarization (EE, BB) and their cross-correlation (TE). We are mainly interested in the TT, TE, and EE power spectra since these are well measured by the *Planck* or WMAP satellite experiments.

In [ ]:
Cl_TT_LCDM = cls_LCDM['tt'][2:]
Cl_TE_LCDM = cls_LCDM['te'][2:]
Cl_EE_LCDM = cls_LCDM['ee'][2:]
ll_LCDM = cls_LCDM['ell'][2:]

It is common to plot the power spectrum multiplied by a $\ell(\ell+1)$-pre-factor: $D_\ell = \frac{\ell (\ell+1)}{2\pi} C_\ell$

Let's plot the $D_\ell$ power spectrum in TT:

In [ ]:
plt.title("TT")
plt.plot(ll_LCDM, ll_LCDM*(ll_LCDM+1)*Cl_TT_LCDM/(2*np.pi))
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\ell(\ell+1)\,C^\mathrm{TT}_\ell/(2\pi)$")
plt.rcParams["figure.figsize"] = (7,5)

Now, we want to compare our theory predictions of the CMB power spectrum to real *Planck* data. For that, let's first have a look at the [*Planck* 2018 paper](https://arxiv.org/pdf/1807.06209.pdf). In Fig. 1 of the paper, you can see the TT power spectrum.

The shape of the TT power spectrum we plotted with CLASS looks good but the $y$-axis is off. This is because in the *Planck* paper, they use $\mu$K units while CLASS outputs the dimensionless power spectrum.

Let's get the CMB temperature from CLASS and
save the $D_\ell$'s in $\mu$K units:

In [ ]:
T_CMB = LCDM.T_cmb() * 1e6 # CLASS outputs T_CMB in K, but we want muK
Dl_TT_LCDM = ll_LCDM*(ll_LCDM+1)*Cl_TT_LCDM/(2*np.pi)*T_CMB**2
Dl_TE_LCDM = ll_LCDM*(ll_LCDM+1)*Cl_TE_LCDM/(2*np.pi)*T_CMB**2
Dl_EE_LCDM = ll_LCDM*(ll_LCDM+1)*Cl_EE_LCDM/(2*np.pi)*T_CMB**2

Now, we want to directly compare to the *measured* CMB power spectra. Here, we will use foreground- and nuisance marginalized spectra, i.e. instrument noise and foreground effects from foreground sources like: point sources, cosmic infrared background, Sunyaev-Zeldovic effect, and dust, have been taken into account already.

You can get these files from the `plik_lite` likelihood of ESA's [*Planck* legacy archive](http://pla.esac.esa.int/pla/#home). For your convenience, I also copied them to [my github](https://github.com/LauraHerold/CosmoVerseSchool_Corfu/raw/main/plik_lite/). If you haven't done so already, download them to your computer and adapt the path to the folder below:

In [ ]:
fn ="plik_lite"

Let's read the power spectra from file:

In [ ]:
Cl_matrix = np.loadtxt(fn+"/cl_cmb_plik_v22.dat").T
lls_Planck, Cls_Planck, sigmas_Planck = Cl_matrix

The file is one big vector containing the TT, TE and EE power spectra:

In [ ]:
ll_TT_Planck = lls_Planck[0:215]
Cl_TT_Planck = Cls_Planck[0:215]
sigma_ClTT_Planck = sigmas_Planck[0:215]

ll_TE_Planck = lls_Planck[215:414]
Cl_TE_Planck = Cls_Planck[215:414]
sigma_ClTE_Planck = sigmas_Planck[215:414]

ll_EE_Planck = lls_Planck[414:613]
Cl_EE_Planck = Cls_Planck[414:613]
sigma_ClEE_Planck = sigmas_Planck[414:613]

Let's compute the $D_\ell$'s as well:

In [ ]:
Dl_TT_Planck = ll_TT_Planck * (ll_TT_Planck+1) / (2.*np.pi) * Cl_TT_Planck
Dl_TE_Planck = ll_TE_Planck * (ll_TE_Planck+1) / (2.*np.pi) * Cl_TE_Planck
Dl_EE_Planck = ll_EE_Planck * (ll_EE_Planck+1) / (2.*np.pi) * Cl_EE_Planck

And rescale the errorbars:

In [ ]:
sigma_TT_Planck = ll_TT_Planck * (ll_TT_Planck+1) / (2.*np.pi) * sigma_ClTT_Planck
sigma_TE_Planck = ll_TE_Planck * (ll_TE_Planck+1) / (2.*np.pi) * sigma_ClTE_Planck
sigma_EE_Planck = ll_EE_Planck * (ll_EE_Planck+1) / (2.*np.pi) * sigma_ClEE_Planck

Let's plot both the TT power spectra measured by *Planck* and our theory predictions that we computed with CLASS:

In [ ]:
plt.title("TT")
plt.plot(ll_LCDM, Dl_TT_LCDM, label="Theory prediction CLASS")
plt.errorbar(ll_TT_Planck, Dl_TT_Planck, sigma_TT_Planck, ls="", marker=".", label=r"Planck data")
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\ell(\ell+1)\,C^\mathrm{TT}_\ell/(2\pi)$")
plt.rcParams["figure.figsize"] = (7,5)
plt.legend()
plt.show()

Let's also plot the TE polarization power spectra and compare them to *Planck* data:

In [ ]:
plt.title("TE")
plt.plot(ll_LCDM, Dl_TE_LCDM, label="Theory prediction CLASS")
plt.errorbar(ll_TE_Planck, Dl_TE_Planck, sigma_TE_Planck, ls="", marker=".", label=r"Planck data")
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\ell(\ell+1)\,C^\mathrm{TE}_\ell/(2\pi)$")
plt.rcParams["figure.figsize"] = (7,5)
plt.legend()
plt.show()

And the same for EE:

In [ ]:
plt.title("EE")
plt.plot(ll_LCDM, Dl_EE_LCDM, label="Theory prediction CLASS")
plt.errorbar(ll_EE_Planck, Dl_EE_Planck, sigma_EE_Planck, ls="", marker=".", label=r"Planck data")
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\ell(\ell+1)\,C^\mathrm{EE}_\ell/(2\pi)$")
plt.rcParams["figure.figsize"] = (7,5)
plt.legend()
plt.show()

The theory predictions fit the *Planck* data points remarkably well!

This is because we used the $\Lambda$CDM mean parameters obtained in the [*Planck* 2018 paper](https://arxiv.org/pdf/1807.06209.pdf) (second column of Table 1). In the following, we want to explore how the theory prediction changes when changing certain $\Lambda$CDM parameters.

**<font color='Crimson'>(a) Exercise:</font>** In the two cells below you will write your own code to explore the effect of different cosmological parameters. Follow these steps:
1.   Initialize a new Class object "test" with `test = Class()`
2.   Set the LCDM parameters to the *Planck* bestfit cosmology as we did above
3.   However, instead of setting 'h', we will fix the angular scale of the sound horizon, $\theta_s = \frac{r_s}{D_A(z^*)}$, to the Planck bestfit value. So remove the line `'h':0.6732,` and add the line `'100*theta_s':1.04092`.
4.   Compute the $C_\ell$'s and convert them to $D_\ell$'s.
5.   In the lower cell, use `matplotlib` as above to plot the `LCDM` spectra using the label "Planck bestfit cosmology" and your new `test` spectra with the label "Test cosmology". Check that they agree!



In [ ]:
# Compute the Dl_TT_test with CLASS

In [ ]:
# Plot the Planck bestfit cosmology along with the modified LCDM cosmology that you defined above

**<font color='Crimson'>(b) Exercise:</font>** Now, that you have your code set up above, we will explore how certain $\Lambda$CDM parameters impact the CMB TT-power spectrum. For that, change **one parameter at a time** to a lower and a higher value than the *Planck* bestfit. Describe in words what you observe!

How do the following parameters impact the CMB TT-power spectrum:
- The cold-dark matter fraction $\omega_\mathrm{cdm}$?
- The baryon fraction $\omega_b$?
- The amplitude of the primordial power spectrum $A_s$?
- The tilt of the primordial power spectrum $n_s$?

Make notes of what you find!

# Part 2 - EDE

Let's add some exotic component to the universe that might be able to solve the Hubble tension:

In [ ]:
EDE = Class()
EDE.set({'fEDE': 0.1271,
         'log10z_c': 3.563,
         'thetai_scf': 2.768,
         'n_scf':3})

This time we initialised a Class object and also set the EDE parameters:
- $f_\mathrm{EDE}$: the fraction of EDE at $z_c$
- $\log_{10}z_c$: the redshift at which the EDE field starts oscillating
- $\theta_{i, \mathrm{scf}}$: the initial value of the scalar field in the potenital
- $n_\mathrm{scf}$: the index of the scalar field potential, $V(\phi) = V_0 (1-\cos(\phi/f))^n$

The parameters we set here are the bestfit parameters to CMB and galaxy clustering data from [here](https://arxiv.org/pdf/2210.16296).

We will also set the $\Lambda$CDM parameters and some technical parameters:

In [ ]:
fixed_cosmo_params = {'h': 0.7215,
         'A_s': 2.155e-9,
         'n_s':0.9917,
         'omega_b': 0.02278,
         'omega_cdm': 0.1324,
         'tau_reio': 0.05682,
         'N_ncdm': 1,
         'N_ur': 2.0328,
         'Omega_Lambda':0,
         'Omega_fld':0,
         'Omega_scf':-1,
         'scf_parameters':'1, 1, 1, 1, 1, 0.0',
         'scf_tuning_index':3,
         'attractor_ic_scf':'no',
         'CC_scf':1,
         'output':'tCl,pCl,lCl,mPk',
         'lensing':'yes'}
EDE.set(fixed_cosmo_params)

Let's compute the EDE cosmology:

In [ ]:
EDE.compute()

If this didn't work, make sure you are currently using `CLASS_EDE` and not `CLASS`.

Remember yesterday in "Hands-on session 1", when we computed the energy densities of the different components of the universe as a function of redshift? We can do the same with EDE:

In [ ]:
baEDE = EDE.get_background()
baEDE.keys()

Since `rho_scf` contains both EDE and DE (i.e. a cosmological constant $\Lambda$), we need to compute $\Lambda$ independently and subract it from the scalar field to obtain the EDE contribution. Let's make a nice plot:

In [ ]:
plt.figure()
plt.xscale('log')
plt.yscale('log')
plt.xlim([0.1,2e4])
plt.ylim([5e-10, 3e6])
plt.xlabel('$z$')
plt.ylabel(r'$\rho(z)$')
plt.plot(baEDE['z'],baEDE['(.)rho_tot'], c='black', label='total')
rho_Lambda = baEDE['(.)rho_scf'][-1]
plt.plot(baEDE['z'],baEDE['(.)rho_scf']-rho_Lambda,c='teal', label='EDE')
plt.plot(baEDE['z'],baEDE['(.)rho_cdm']+baEDE['(.)rho_b'], c='blue', label='matter', ls="--")
plt.plot(baEDE['z'],baEDE['(.)rho_ur']+baEDE['(.)rho_g'], c='darkorange', label='radiation', ls="--")
plt.axhline(rho_Lambda, label=r"$\Lambda$", color="red", ls="--")
plt.legend()
plt.show()

We can see that EDE is only non-negligible before $z\approx 10^3$, which is around recombination, i.e. the time when the CMB was emitted.

Let's plot the fractional energy density of EDE to see this more clearly:

In [ ]:
plt.figure()
plt.xscale('log')
plt.xlim([1,2e4])
plt.ylim([0,0.13])
plt.xlabel('$z$')
plt.ylabel(r'$\Omega_{{\rm EDE}}(z)/\Omega_\mathrm{tot}(z)$')
plt.plot(baEDE['z'],(baEDE['(.)rho_scf']-rho_Lambda)/baEDE['(.)rho_tot'],c='teal', label=r'EDE')
plt.legend(loc="upper left")
plt.show()

At high redshifts, EDE's contribution to the energy density grows rapidly, then it peaks at the critical redshift $z_c \approx 4\cdot 10^3$ and decays after in a few oscillations.

Let's also compute the CMB power spectra for the EDE model:

In [ ]:
cls_EDE = EDE.lensed_cl(2500)
Cl_TT_EDE = cls_EDE['tt'][2:]
Cl_TE_EDE = cls_EDE['te'][2:]
Cl_EE_EDE = cls_EDE['ee'][2:]
ll_EDE = cls_EDE['ell'][2:]

In [ ]:
Dl_TT_EDE = ll_EDE*(ll_EDE+1)*Cl_TT_EDE/(2*np.pi)*T_CMB**2
Dl_TE_EDE = ll_EDE*(ll_EDE+1)*Cl_TE_EDE/(2*np.pi)*T_CMB**2
Dl_EE_EDE = ll_EDE*(ll_EDE+1)*Cl_EE_EDE/(2*np.pi)*T_CMB**2

And compare them to the *Planck* TT power spectrum:

In [ ]:
plt.title("TT")
plt.plot(ll_EDE, Dl_TT_EDE, label="EDE model", color="green")
plt.errorbar(ll_TT_Planck, Dl_TT_Planck, sigma_TT_Planck, ls="", marker=".", label=r"Planck data", color="tab:orange")
plt.xlabel(r"$\ell$")
plt.ylabel(r"$\ell(\ell+1)\,C^\mathrm{TT}_\ell/(2\pi)$")
plt.rcParams["figure.figsize"] = (7,5)
plt.legend()
plt.show()

By eye, the EDE model fits the TT spectrum as well as $\Lambda$CDM above.

**<font color='Crimson'>(c) Exercise:</font>** EDE was constructed in such a way as not to change the fit to the CMB power spectra much but still achieve a higher $H_0$. Compute $h=H_0/$ (100 km/s/Mpc) for the $\Lambda$CDM and EDE models in the cell below to verify that $H_0$ increased!

In [ ]:
# Compute h_LCDM and h_EDE

# Part 3 - Analysing $\Lambda$CDM and EDE with CMB and supernova data

Now, we want to quantify how well our theory predictions fit the real data. One measure of goodness of fit is the $\chi^2$-function:

$$\chi^2_\mathrm{Planck} = \sum_i (C_\ell^\mathrm{theory} - C_\ell^\mathrm{data})^T \Sigma^{-1} (C_\ell^\mathrm{theory} - C_\ell^\mathrm{data}),$$

where $\Sigma$ is the covariance matrix of the data.

To compute the $\chi^2$, we need the covariance matrix. The covariance matrix is published within the `plik_lite` likelihood of ESA's [*Planck* legacy archive](http://pla.esac.esa.int/pla/#home) but I also copied them to [my github](https://github.com/LauraHerold/CosmoVerseSchool_Corfu/raw/main/plik_lite/). You probably downloaded the covariance matrix "c_matrix_plik_v22.dat" along with the power spectra earlier.

The covariance matrix is saved as an upper diagonal matrix, so we need to symmetrize it:

In [ ]:
dt = np.dtype([('header', np.int32, 1), ('cov', np.float64, (375769,))])
temp = np.fromfile(fn+"/c_matrix_plik_v22.dat", dtype=dt, count=1)
cov = temp['cov'].copy().reshape(613,613)
for i in range(613):
            for j in range(i,613):
                cov[i,j] = cov[j,i]

This covariance matrix is one big matrix containing the covariance of the TT, TE, and EE CMB power spectra.

We will need to save our theory-$C_\ell$'s in the same format:

In [ ]:
ll_TT_int = ll_TT_Planck.astype(int)-2
ll_TE_int = ll_TE_Planck.astype(int)-2
ll_EE_int = ll_EE_Planck.astype(int)-2
Cls_LCDM = np.append(np.append(Cl_TT_LCDM[ll_TT_int], Cl_TE_LCDM[ll_TE_int]), Cl_EE_LCDM[ll_EE_int])*T_CMB**2
Cls_EDE = np.append(np.append(Cl_TT_EDE[ll_TT_int], Cl_TE_EDE[ll_TE_int]), Cl_EE_EDE[ll_EE_int])*T_CMB**2

To compute the $\chi^2$, we need to invert the covariance matrix:

In [ ]:
inv_cov = np.linalg.inv(cov)

Let's write a function that computes the $\chi^2$ of *Planck* data:

In [ ]:
def chi2_Planck(Cls_theo):
  chi2 = np.matmul(np.matmul((Cls_theo-Cls_Planck).T, inv_cov), (Cls_theo-Cls_Planck))
  return chi2

And compute the $\chi^2$ of the $\Lambda$CDM model with parameters obtained by *Planck*:

In [ ]:
print(chi2_Planck(Cls_LCDM))

**<font color='Crimson'>(d) Exercise:</font>** Use the function `chi2_Planck()` to compute the $\chi^2$ of the EDE model in the cell below!

In [ ]:
# compute chi2_EDE_Planck

A smaller $\chi^2$ means a better fit to the data. You should find a difference of a few: $\Delta\chi^2 \sim \mathcal{O}(1)$, which is not significant.

Remember that EDE was constructed in such a way as not to change the fit to the CMB data significantly but still achieve a higher $H_0$. Hence, to see how EDE resolves the $H_0$ tension, we need to include a direct measurement of $H_0$. The most competitive direct measurement to date uses Cepheid-calibrated type-Ia supernovae. The [2022 paper by the SH0ES collaboration](https://arxiv.org/pdf/2112.04510) uses this techique.

**<font color='Crimson'>(e) Exercise:</font>** In this exercise you will write a function to compute the $\chi^2$ of the SH0ES-$H_0$ measurment.


1.   Open the paper by the SH0ES collaboration, which is linked above, and find $H_0$ its 1$\sigma$-errorbar $\sigma_{H_0}$ (use the baseline result).
2.   Define the dimensionless parameter $h=H_0/(100\,\mathrm{km/s/Mpc})$. Note that you also have to rescale $\sigma_h = \sigma_{H_0}/(100\,\mathrm{km/s/Mpc})$
3.   Since this is a direct measurement of a single parameter, there is no correlation, i.e. no covariance matrix, and the $\chi^2$ simplifies to
$$ \chi^2_h = \frac{(h_\mathrm{data} - h_\mathrm{theory})^2}{\sigma_h^2}$$
Write a function `chi2_SH0ES(h_theory)` that computes $\chi^2_h$ for a given $h$ obtained from theory!
4.   Use your function to compute $\chi^2_h$ of $h_{\Lambda\mathrm{CDM}}$ and $h_\mathrm{EDE}$!



In [ ]:
# Define chi2_SH0ES(h)

In [ ]:
# Compute chi2_SH0ES(LCDM.h()) and chi2_SH0ES(EDE.h())

One may now ask the question which model presents the better fit to the data, is it $\Lambda$CDM or EDE? One simple approach to answer this question is by considering the difference in $\chi^2$:
$$ \Delta\chi^2 = \chi^2_{\Lambda\mathrm{CDM}}-\chi^2_\mathrm{EDE}$$
Since the models are nested, i.e. the $\Lambda$CDM model is a subset of the EDE model (namely for $f_\mathrm{EDE}=0$), one can make use of Wilks' theorem:

The $\Delta\chi^2$ is distributed as a $\chi^2$-distribution with $n$ degrees of freedom, where $n$ is the number of extra parameters in the model with more parameters, i.e. here $n=3$. E.g. for $n=3$, $\Delta\chi^2 \approx 14$ corresponds to $3\sigma$ significance, $\Delta\chi^2\approx 22$ corresponds to $4\sigma$, and $\Delta\chi^2 \approx 32$ corresponds to $5\sigma$, etc.

**<font color='Crimson'>(e) Exercise:</font>** Compute $\Delta\chi^2$ as defined above!

In [ ]:
# Compute Delta chi^2

**<font color='red'>WARNING:</font>** These results should be taken as a toy example. The cosmologies for $\Lambda$CDM and EDE that we are using here were obtained from a different data set (i.e. the full *Planck* likelihood) than the one we are using here to compute the goodness of fit.

**Conclusions:** In this exercise, we learned how to use `CLASS` and its EDE extension to predict the CMB power spectra, the time evolution of energy densities and the Hubble constant. We used this to quantify the goodness of fit to CMB data and a direct measurement of $H_0$. We saw how EDE can improve the goodness of fit to these data sets.




# Additional optional exercises

Here are some additional exercises, which you can do if you finish early or at home.

**<font color='Crimson'>(f) Exercise:</font>** Explore how different values of
- $f_\mathrm{EDE}$
- $\log_{10}z_c$
- $\theta_{i, \mathrm{scf}}$
- $n_\mathrm{scf}$

impact $\Omega_\mathrm{EDE}$ by varying one parameter at a time. Take notes of what you find!

**<font color='Crimson'>(g) Exercise:</font>** Install the codes you need for the [next hands-on session](https://gitlab.com/matmartinelli/cosmoverse_school).

**<font color='Crimson'>(h) Exercise:</font>** Use the tools introduced in this lecture to resolve the Hubble tension!